In [2]:
import asyncio
from pynq import Overlay, allocate, Interrupt
import numpy as np

# Load overlay
ol = Overlay("sistema.bit")

# Configure your buffers and IP as usual
size = 8192
input_buffer = allocate(shape=(size,), dtype=np.float32)
output_buffer = allocate(shape=(size,), dtype=np.float32)
input_buffer[:] = np.random.rand(size).astype(np.float32)
gain = np.float32(2.0)

kr = ol.array_gain_0
kr.write(kr.register_map.input_r_1.address, input_buffer.physical_address)
kr.write(kr.register_map.output_r_1.address, output_buffer.physical_address)
gain_int = int(np.frombuffer(gain.tobytes(), dtype=np.uint32)[0])
kr.write(kr.register_map.gain.address, gain_int)

# Instantiate the interrupt using the name from the block design
irq = Interrupt("axi_intc_0/intr")

# Define an asynchronous coroutine to wait for the interrupt
async def wait_for_interrupt():
    await irq.wait()  # This will block until the interrupt is triggered
    print("Interrupt received! Kernel computation completed.")
    # Optionally, print results or process buffers
    print("First 10 input values :", input_buffer[:10])
    print("First 10 output values:", output_buffer[:10])
    print("Gain used              :", gain)
    irq.clear()  # Clear the interrupt flag

# Schedule the asynchronous interrupt wait
asyncio.get_event_loop().create_task(wait_for_interrupt())

# Start the kernel computation
kr.register_map.CTRL.AP_START = 1
